<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Complete_LSTM_Training_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

# --- Model ---
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Last time step

# --- Hyperparameters ---
seq_length = 5
batch_size = 16
input_dim = 4
hidden_dim = 32
output_dim = 2
num_epochs = 10
lr = 0.001

# --- Synthetic Data ---
X = torch.rand(500, seq_length, input_dim)
y = torch.randint(0, output_dim, (500,))  # Class indices for classification

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# --- Model, Loss, Optimizer ---
model = SimpleLSTM(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# --- Training Loop ---
for epoch in range(num_epochs):
    total_loss = 0
    for batch_x, batch_y in loader:
        output = model(batch_x)
        loss = criterion(output, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# --- Output Shape Check ---
with torch.no_grad():
    sample_input = torch.rand(batch_size, seq_length, input_dim)
    sample_output = model(sample_input)
    print("LSTM Output Shape:", sample_output.shape)  # Should be [batch_size, output_dim]